<a href="https://colab.research.google.com/github/BerkMandiracioglu/AppliedDataAnalysisHW1/blob/b-part-sena/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1
## Introduction
### Important dates
* Homework release: Thursday, 3rd October 2019
* **Homework due**: Wednesday, 16th October 2019 (23:59 hours, 11:59 PM)
* Peer review due: Wednesday, 23rd October 2019 (23:59 hours, 11:59 PM)
* Grading release: Wednesday, 30th October 2019

### Description

The data you'll be working with comes from multiple sources. The main data source will be [DBLP](https://dblp.uni-trier.de/), a database of publications from major computer science journals and conferences. A subset of DBLP, which you will use in this assignment, is provided to you via a [google drive folder](https://drive.google.com/file/d/1Kci8joML74tCSzuBbhxtd1ylR4f0dlm6/view). Later on, you will enrich the DBLP data with a dataset on conference rankings and with the proceedings of the [NIPS conference](https://nips.cc/) [1] ('proceedings' is another word for the set of papers published at an academic conference). After loading and cleaning the data, you will answer various questions about its contents.

**Some rules:**
- You are allowed to use any built-in Python library that comes with Anaconda. If you want to use an external library, you have to justify your choice.
- Make sure you use the data folder provided in the repository in *read-only* mode.
- Be sure to provide explanations for your answers. A notebook that only has code cells will not suffice.
- Also, be sure to *hand in a fully-run and evaluated notebook*. We will not run your notebook for you, we will grade it as is, which means that only the results contained in your evaluated code cells will be considered, and we will not see the results in unevaluated code cells. In order to check whether everything looks as intended, you can check the rendered notebook on the GitHub website once you have pushed your solution there.

[1] Note that NIPS was renamed to NeurIPS in 2018, but for simplicity, whenever we say 'NIPS', we really mean 'NIPS and NeurIPS'.

## Task A. Getting a sense of the dataset

### A1. Loading the data
Download the DBLP dataset (available on [google drive](https://drive.google.com/file/d/1Kci8joML74tCSzuBbhxtd1ylR4f0dlm6/view)) and load it into a Pandas dataframe. A row of your dataframe should look as follows:

| paper id | author names | publication year | paper title | 
| :----:|:-------------:| :-----:|:-----:|
| conf/nips/doe1036 | [John Doe, Jane Doe] | 2003 | Some Catchy Title: An Expanded and Boring Title | 


1. Filter the papers: keep only conference papers. For each of the remaining ones, find the acronym of the conference where it was published. Retain only those papers that have been published in the conferences listed in `data/list_of_ai_conferences.txt`. Additionally, add a column named 'conference' to your dataframe.   
_Hint: The `paper id` tells you whether a paper was published at a conference, and if so, at which one._

2. Report the overall number of papers in the filtered dataset, as well as the number of papers per conference.

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
#from requests import get
#from bs4 import BeautifulSoup

In [2]:
from google.colab import files
uploaded = files.upload()

Saving dblp.tsv to dblp.tsv


In [0]:
''' Add your code here'''
#Load the dataset
dblp = pd.read_csv('dblp.tsv', sep='\t')

In [0]:
dblp.rename(columns={"id": "paper id", "authors": "author names", "year": "publication year", "title": "paper title"}, inplace=True)
dblp_papers = dblp[dblp['paper id'].str.contains('^conf.*')]

In [0]:
def extract_conference(name):
    return re.split("/", name)[1]

In [86]:
#Extract the name of the conference and create a new column
conferences = []
for name in dblp_papers["paper id"]:
    conferences.append(extract_conference(name))
dblp_papers.insert(4,"Conference",conferences)
#conferences = ['cvpr','icra','iccv','icml','nips','acl','emnlp','aaai','aamas','ijcai','naacl','accv','iccbr','ida','ilp','pricai','ksem','icann','iconip','fgr']
dblp_papers.reset_index(drop=True, inplace=True)
display(dblp_papers)    

ValueError: ignored

In [8]:
papers_count = dblp_papers["paper title"].nunique() 
print("The total number of papers is: %d" %papers_count)
display(dblp_papers.groupby('Conference')['Conference'].count())

The total number of papers is: 2415276


Conference
3dgis           57
3dic           525
3dica          461
3dim           951
3dor           188
3dph            20
3dpvt          413
3dtv           261
3dui           553
3pgcic         824
5gu             53
5gwf           101
5gwn            64
ACISicis      1909
ACMace        1128
ACMdis        1290
ACMicec        727
ACMmsp          31
ACMpolicy       21
ACMse         1879
Dstep           49
EUprojects      53
IEEEants       620
IEEEares      1783
IEEEcca       2428
IEEEcgiv       534
IEEEcit       2521
IEEEcloud     1205
IEEEcsa         42
IEEEcsc        122
              ... 
wsom           252
wspi            27
wsr             21
wss             46
wssanlp         58
wstfeus         58
wstst          249
wts            629
wuauc           26
wuwnet         388
wwca            68
wwic           428
wwv             30
www           6016
xfree            8
ximep           39
xmidx           11
xml             20
xmlsec          26
xp              89
xps            110
x

### A2. An author-centric look
The dataframe you created above was a paper-centric one. Now, we want you to create a new dataframe centered around authors. Do this by expanding the author names in the lists in the 2nd column into separate rows. That is, if a paper has 3 authors, turn that row into 3 rows, each of which only contains one of the author names (along with the rest of the paper information, i.e., title, conference and year). Keep both dataframes, we are going to need both of them.    
**Report the number of unique authors.**

In [0]:
def delete_parantheses(name):
    output = name
    if(name.startswith('[')):
        output = name[2:]
    if((name.endswith(']'))):
        output = output[:-2]
    return output
    
def delete_parantheses1(name):
    output = re.sub(r'\[^[]]*\]','', name)
    return output

In [0]:
''' Add your code here'''
# Reindex and repeat cols on len of split and reset index.
dblp_authors = dblp_papers.reindex(dblp_papers.index.repeat(dblp_papers['author names'].str.split("', '").apply(len))) 
dblp_authors = dblp_authors.reset_index(drop=True)

In [0]:
#Split on comma, create list of authors and reset index
authors_list = pd.DataFrame(dblp_papers['author names'].str.split("', '").tolist()).stack()
authors_list.reset_index(drop=True, inplace=True)

In [0]:
#Replace columns
dblp_authors['author names'] = authors_list.apply(delete_parantheses)

authors_count = dblp_authors["author names"].nunique() 
print("The total number of authors is: %d" %authors_count)

### A3. Is your data analysis-friendly?

Using an appropriate view of your data (paper-centric or author-centric), solve the following tasks:

1. Plot the number of papers per author and analyze it. Do you observe any outliers? Can you identify the source of the problem? Please elaborate!   
_Hint: To find out where the problem comes from, try performing an analysis at the conference or year level._   
Make sure you remove this inconsistency from your dataframe before moving to the next step, and also create a new plot of the number of papers per author after fixing the problem.   

2. Plot the number of papers per year. Do you observe any inconsistency in the output? Real-world data is usually messy, with typos, erroneous entries, and sometimes issues that make even loading the data problematic. Fix any errors that you encounter along the way, find ways to clean the attribute `year`, and redo the plot of the number of papers per year.   

3. Machine learning (ML) has been one of the hottest topics within the broader area of AI recently, so let’s see if this is reflected in the number of ML publications. In particular, let’s focus on the two major ML conferences, NIPS and ICML: make a new dataframe with only NIPS and ICML papers (let’s call these the “ML papers”), plot the number of ML papers over time, and analyze and discuss the plot. Do you observe anything odd in this plot? What causes these problems?   
_Hint: Try to perform an analysis at the conference or year level._   

4. By now, you may have noticed that some conferences are not fully represented in the DBLP dataset. Complete the paper-centric dataframe by scraping the full NIPS data from the online proceedings at https://papers.nips.cc/ (maintain the same schema used in your previous dataframes, but fill in missing values). After this step, remove any remaining papers that have missing values. Redo the plots of steps A3.2 and A3.3 after fixing the issue.   

_Note: In order to avoid re-running the cleaning part of the notebook every time, you could save the results at this point as a pickle file! Also, propagating your cleaning to both dataframes might prove useful later on._

***A3.1***

In [0]:
''' Add your code here '''
grouped_authors = dblp_authors.groupby("author names").count().reset_index()
display(grouped_authors['author names'] - dblp_authors['author names'])
#plt.bar(grouped_authors.index, grouped_authors["paper id"])
 



***A3.2***

In [0]:
''' Add your code here '''
def filter_year(year):
    digits = re.sub("\D", "", year)
    if(len(digits)>4):
        return digits[0:4]
    elif(len(digits) == 2):
        if(digits[0] == '0' or digits[0] == '1'):
            return '20' + digits
        else:
            return '19' + digits
    else:
        return digits
        


dblp_papers["publication year"] = dblp_papers["publication year"].apply(filter_year)
dblp_authors["publication year"] = dblp_authors["publication year"].apply(filter_year)
display(set(dblp_papers['publication year']))

***A3.3***

In [0]:
mlconf = ['icml', 'nips']
''' Add your code here '''
ml_papers = dblp_papers.copy(deep =True)
ml_papers = dblp_papers[dblp_papers['Conference'].str.contains('icml|nips')]
ml_papers

***A3.4***

In [0]:
''' Add your code here '''

### A4. Author activity

For each author, calculate their normalized paper count, defined as the total number of papers divided by the author’s period of activity. An author’s period of activity is defined as the number of years between the earliest and latest papers of this author. Plot the distribution of the normalized paper count. What is the appropriate scale for the axes? Does the distribution (roughly) follow a particular law, and if yes, which one?

In [0]:
''' Add your code here '''

## Task B. Ranking authors

As you may know, there exist rankings for universities, which represent their relative quality and are used to compare the performance of different universities. In the same vein, there are rankings for conferences and journals, which represent the importance and impact of each conference or journal, and therefore allow for approximate comparisons. In this part, you will rank authors based on different aspects of their research output.

### B1. A Naïve Score

In the absence of citation counts, it is hard to objectively rank the authors based on the impact of their contributions to the field of AI research. A naïve way would be to rank them based on their number of published papers. Obtain such a ranking and analyze your result. Identify and explain some obvious limitations of this scheme.

In [0]:
''' Add your code here '''
grouped = dblp_authors.groupby('author names').count().reset_index()

grouped = grouped.drop(['publication year','paper title','Conference'], axis = 1)
grouped.rename(columns={'paper id':'paper rank'},inplace=True)

grouped = grouped.sort_values('paper rank', ascending=False).reset_index().drop('index',axis=1)


### B2. H5-index

Another way to score and rank authors could be based on the quality of the conferences and journals where they publish their papers. For this task, you have to use the H5-index score from AMiner (https://aminer.org/ranks/conf) (another database of scholarly publications), which captures the quality of academic conferences: the higher the H5-index, the better the conference.
1. Load the AMiner dataset ( *'aminer_ai.tsv'* available in the folder ``data/``), which contains H5-index values for AI conferences. Load it into a new Pandas dataframe, and join it with the author-centric DBLP dataframe.
2. Calculate a *'new'* author ranking (give each author a score, by which the authors are then sorted in order to obtain the ranking), where each author's score is the sum of the H5-indices of all their papers (the H5-index of a paper being the H5-index of the conference it is published in).
3. Analyze your new, H5-index-based author ranking and explain how and why your results are different from the previous ranking. Do you see any differences in the top-20 authors based on the H5-index-based ranking and the one produced using publication counts? If yes, list the authors that are ranked in the top 20 based on publication counts but absent in the top 20 based on the H5-index-based ranking. Identify the ranks of these authors in the ranking produced by the H5-index based ranking scheme.
4. Now, take the authors in the file `data/list_of_selected_authors.txt`, and compute their rankings using the two (naïve and H5-index-based) ranking schemes. What do you observe? Explain the potential dangers of the naïve, paper-count-based score.
5. On the flip side, do you see any potential dangers of using the H5-index-based score?   
_Hint: Analyze the conferences in which the top ranked authors publish. Investigate the effect of the conferences in which these authors publish more frequently on the obtained ranking._

***B2.1***

In [171]:
''' Add your code here '''
aminer = pd.read_csv('aminer_ai.csv', sep=';')
aminer["Short Name"] = aminer["Short Name"].str.lower()
merged_aminer = pd.merge(aminer, dblp_authors, left_on='Short Name', right_on='Conference')
merged_aminer

,Rank,Conference (Full Name),Short Name,H5-Index,paper id,author names,publication year,paper title,Conference
0,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/BonetV98,"['Jeremy S. De Bonet', 'Paul A. Viola']",1998.0,Texture Recognition Using a Non-Parametric Mul...,cvpr
1,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/BonetV98,"['Jeremy S. De Bonet', 'Paul A. Viola']",1998.0,Texture Recognition Using a Non-Parametric Mul...,cvpr
2,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/KonishiYCZ99,"['Scott Konishi', 'Alan L. Yuille', 'James M. ...",1999.0,Fundamental Bounds on Edge Detection: An Infor...,cvpr
3,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/KonishiYCZ99,"['Scott Konishi', 'Alan L. Yuille', 'James M. ...",1999.0,Fundamental Bounds on Edge Detection: An Infor...,cvpr
4,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/KonishiYCZ99,"['Scott Konishi', 'Alan L. Yuille', 'James M. ...",1999.0,Fundamental Bounds on Edge Detection: An Infor...,cvpr
5,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/KonishiYCZ99,"['Scott Konishi', 'Alan L. Yuille', 'James M. ...",1999.0,Fundamental Bounds on Edge Detection: An Infor...,cvpr
6,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/RennoMJ07,"['John-Paul Renno', 'Dimitrios Makris', 'Graem...",<em>2007.0</em>,Object Classification in Visual Surveillance U...,cvpr
7,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/RennoMJ07,"['John-Paul Renno', 'Dimitrios Makris', 'Graem...",<em>2007.0</em>,Object Classification in Visual Surveillance U...,cvpr
8,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/RennoMJ07,"['John-Paul Renno', 'Dimitrios Makris', 'Graem...",<em>2007.0</em>,Object Classification in Visual Surveillance U...,cvpr
9,1,IEEE Conference on Computer Vision and Pattern...,cvpr,112,conf/cvpr/ZhangWBLL18,"['Shifeng Zhang', 'Longyin Wen', 'Xiao Bian', ...",'18,Single-Shot Refinement Neural Network for Obje...,cvpr


***B2.2***

In [0]:
''' Add your code here '''
rankings = merged_aminer.groupby('author names')[['H5-Index']].sum()
rankings = rankings.sort_values('H5-Index', ascending=False).reset_index()
rankings.rename(columns={'H5-Index':'paper rank'},inplace=True)

In [173]:
rankings

,author names,paper rank
0,[],47961
1,"['Radu Timofte', 'Eirikur Agustsson', 'Luc Van...",8624
2,"['Ole Johannsen', 'Katrin Honauer', 'Bastian G...",3136
3,"['Shanxin Yuan', 'Guillermo Garcia-Hernando', ...",2688
4,"['Sangmin Oh', 'Anthony Hoogs', 'A. G. Amitha ...",2688
5,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",2480
6,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",2224
7,"['Dario Amodei', 'Sundaram Ananthanarayanan', ...",2184
8,"['Takayuki Okatani', 'Koichiro Deguchi']",2088
9,"['Quanshi Zhang', 'Xuan Song', 'Xiaowei Shao',...",1990


***B2.3***

In [174]:
''' Add your code here '''
top_rankings = rankings.head(20).reset_index()
top_grouped = grouped.head(20).reset_index()

pd.concat([top_grouped, top_rankings,top_rankings]).drop_duplicates(subset='author names',keep=False)

,index,author names,paper rank
1,1,"['Irith Pomeranz', 'Sudhakar M. Reddy']",348
2,2,"['Devis Bianchini', 'Valeria De Antonellis', '...",156
4,4,"['Manuel Clavel', 'Francisco Durán', 'Steven E...",147
5,5,"['Shusaku Tsumoto', 'Shoji Hirano']",142
7,7,"['Narasimha R. Adiga', 'George Almási 0001', '...",115
8,8,"['Yoshiki Kawata', 'Noboru Niki', 'Hironobu Oh...",110
9,9,"['Shoji Hirano', 'Shusaku Tsumoto']",106
10,10,['Ichiro Satoh'],106
12,12,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",104
13,13,"['Ian T. Foster', 'Jerry Gieraltowski', 'Scott...",102


The results obtained from using the H5-index-based author ranking differ from the ranking obtained by the sum of the publication counts. This difference is due to giving a higher ranking to authors who have published their papers in conferences with higher H5-indices. 

*Analyze your new, H5-index-based author ranking and explain how and why your results are different from the previous ranking. Do you see any differences in the top-20 authors based on the H5-index-based ranking and the one produced using publication counts? If yes, list the authors that are ranked in the top 20 based on publication counts but absent in the top 20 based on the H5-index-based ranking. Identify the ranks of these authors in the ranking produced by the H5-index based ranking scheme.*

***B2.4***

In [175]:
''' Add your code here '''
lines = open("list_of_selected_authors.txt", 'r')
lines = lines.read().split('\n')

rankings.loc[rankings['author names'].isin(lines)]

,author names,paper rank


*Write on "what you observe" once you obtain the actual data*

https://bitesizebio.com/13614/does-your-h-index-measure-up/

The naive, paper-count-based score is not a good indicator of the ranking of an author. The number of papers published gives no indication of their impact on the field. A small number of publications that have been published in a good-quality conference are better than a long list of publications cited poorly, or not at all.

***B2.5***

The H-indices of the conferences differ vastly. For example, CVPR, which is the highest H-indexed conference, has an H-index of 112, whereas Expert Syst. Appl., which is the third highest H-indexed conference, has an H-index of 58. This shows that even a little difference in the ranking of the conferences can mean a large difference in their H-indices, which may lead to a large difference in the ranking of the authors. 

Moreover, some conferences are very specific to certain topics. For example, CVPR, which is the conference with the highest H-index, is on Computer Vision and Pattern Recognition, TPAMI, which is the conference with the second highest H-index, is on pattern analysis and machine intelligence. So, if an author does not publish a paper in the fields of these two conferences, they become extremely disadvantaged.

*Give concrete examples after obtaining the actual data*

### B3. And Justice For All

An ideal ranking scheme should not give undue advantage to authors who have been conducting research for a longer period of time and therefore have naturally published more papers when compared to a junior researcher. Does the ranking scheme designed by you in ``Step 2`` take this factor into account? If not, introduce variations in your ranking scheme to mitigate this effect. Do you observe anything odd with this new ranking? Clearly explain your observations.

_Hint: What you did in part A4 may be useful here._

The H5-based ranking scheme we designed in Step 2 does not take into account the number of years the author has been conducting research.

(H5-index of conference) - (years passed since paper has been published)

author's points = sum over all papers ((H5 index of the conference the paper has been published in) - (years passed since the paper has been published))

In [0]:
''' Add your code here '''
def filter_year(year):
    digits = re.sub("\D", "", year)
    if(len(digits)>4):
        return digits[0:4]
    elif(len(digits) == 2):
        if(digits[0] == '0' or digits[0] == '1'):
            return '20' + digits
        else:
            return '19' + digits
    else:
        return digits

In [0]:
# Regularize publication year values
merged_aminer["publication year"] = merged_aminer["publication year"].apply(filter_year)

In [0]:
# Calculate number of years passed since the papers have been published
merged_aminer["publication year"] = pd.to_numeric(merged_aminer["publication year"])
merged_aminer["publication year"] = 2019 - merged_aminer["publication year"]
merged_aminer.rename(columns = {"publication year": "years passed"}, inplace=True)

In [179]:
# Create new ranking scheme by subtracting the quarter of the number of years passed since published from the H5-index of the conference 
merged_aminer['H5_years'] = merged_aminer['H5-Index'] - merged_aminer['years passed']/4
fair_rankings = merged_aminer.groupby('author names')[['H5_years']].sum()
fair_rankings = fair_rankings.sort_values('H5_years', ascending=False).reset_index()
fair_rankings.rename(columns={'H5_years':'paper rank'}, inplace=True)
fair_rankings.head(5)

,author names,paper rank
0,[],43023.5
1,"['Radu Timofte', 'Eirikur Agustsson', 'Luc Van...",8585.5
2,"['Ole Johannsen', 'Katrin Honauer', 'Bastian G...",3122.0
3,"['Shanxin Yuan', 'Guillermo Garcia-Hernando', ...",2682.0
4,"['Sangmin Oh', 'Anthony Hoogs', 'A. G. Amitha ...",2640.0


In [180]:
fair_rankings

,author names,paper rank
0,[],43023.50
1,"['Radu Timofte', 'Eirikur Agustsson', 'Luc Van...",8585.50
2,"['Ole Johannsen', 'Katrin Honauer', 'Bastian G...",3122.00
3,"['Shanxin Yuan', 'Guillermo Garcia-Hernando', ...",2682.00
4,"['Sangmin Oh', 'Anthony Hoogs', 'A. G. Amitha ...",2640.00
5,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",2441.25
6,"['Dario Amodei', 'Sundaram Ananthanarayanan', ...",2154.75
7,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",2119.75
8,"['Takayuki Okatani', 'Koichiro Deguchi']",1970.00
9,"['Quanshi Zhang', 'Xuan Song', 'Xiaowei Shao',...",1956.25


In [0]:
# Convert negative rank values to 0
mask = fair_rankings['paper rank'] < 0
column_name = 'paper rank'
fair_rankings.loc[mask, column_name] = 0

In [182]:
fair_rankings

,author names,paper rank
0,[],43023.50
1,"['Radu Timofte', 'Eirikur Agustsson', 'Luc Van...",8585.50
2,"['Ole Johannsen', 'Katrin Honauer', 'Bastian G...",3122.00
3,"['Shanxin Yuan', 'Guillermo Garcia-Hernando', ...",2682.00
4,"['Sangmin Oh', 'Anthony Hoogs', 'A. G. Amitha ...",2640.00
5,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",2441.25
6,"['Dario Amodei', 'Sundaram Ananthanarayanan', ...",2154.75
7,"['Matej Kristan', 'Ales Leonardis', 'Jiri Mata...",2119.75
8,"['Takayuki Okatani', 'Koichiro Deguchi']",1970.00
9,"['Quanshi Zhang', 'Xuan Song', 'Xiaowei Shao',...",1956.25


## Task C. Trending topics

Historically, the field of AI has witnessed research in two broad flavors: “symbolic” (logic, planning, control, etc.) vs. “connectionist” (neural networks, deep learning, Bayesian methods, etc.). Let’s see if we can see how the popularity of these two approaches to AI is reflected in the DBLP data.

To this end, construct two dataframes: ``symbolic`` and ``connectionist``. ``symbolic`` is your paper-centric dataframe from part A1 filtered down to those papers whose titles contain at least one of the following words (not differentiating between upper and lower case letters): “logic”, “planning”, “control”; ``connectionist`` is a dataframe constructed in a similar manner, but with the words “deep”, “learning”, “feature”, “bayesian”. Plot the number of papers per year for ``symbolic`` and ``connectionist`` separately (i.e., 2 plots).
1. Describe the trends you observe. Based on these plots alone, what might one conclude about the popularity of the two approaches to AI?
2. Moving beyond these plots, what do you, as a careful data scientist, conclude about the popularity of symbolic vs. connectionist AI? Corroborate your reasoning with further plots.

_Note: You could use the text handling utilities below to clean the text in the paper titles._

In [0]:
# Text handling utilities
from string import punctuation
stopwords_list = open('data/stopwords.txt', 'r').readlines()
stopwords_list = [x.strip() for x in stopwords_list]
def stopword_remover(text):
    text_list = text.split()
    text_list = [x for x in text_list if x not in stopwords_list]
    return ' '.join(text_list)
def lowercase_all(text):
    return text.lower()
def remove_punct(text):
    return ''.join([ch for ch in text if ch not in punctuation])

In [0]:
''' Add your code here '''

In [0]:
words_symbolic = ['logic', 'planning', 'control']
''' Add your code here '''

In [0]:
words_connectionist = ['deep', 'learning', 'feature', 'bayesian']
''' Add your code here '''